In [4]:
import os
import random
import matplotlib.pyplot as plt
import seaborn as sns
from os.path import join
from tqdm import tqdm
import pandas as pd
from typing import List
import numpy as np
import sasc.generate_helper
import sasc.viz
from pprint import pprint
import joblib
from collections import defaultdict
from sasc.config import RESULTS_DIR, REPO_DIR, CACHE_DIR
from typing import Tuple
import sys
import json

sys.path.append(join(REPO_DIR, "notebooks_stories", "0_voxel_select"))
import pilot

# set openai key
import openai

openai.api_key_path = "/home/chansingh/.OPENAI_KEY"

### Select voxels
Based on their similarity to predicted ngrams

In [5]:
subject = "UTS02"
cache_ngrams = joblib.load(join(CACHE_DIR, 'cache_ngrams', f'fmri_{subject}.pkl'))

In [6]:
rows = pilot.get_rows_voxels(subject=subject)

# drop 7 voxels with worse performance
worst_voxels = [368, 342, 248, 179, 155, 154, 168]
rows = rows[~rows.module_num.isin(worst_voxels)]
cache_ngrams_mod = cache_ngrams[:, rows.module_num.values]

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 rows = pilot.get_rows_voxels(subject=subject)                                                │
│   2                                                                                              │
│   3 # drop 7 voxels with worse performance                                                       │
│   4 worst_voxels = [368, 342, 248, 179, 155, 154, 168]                                           │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: module 'pilot' has no attribute 'get_rows_voxels'

In [ ]:
# get correlations of every column with every other column
corrs = np.corrcoef(cache_ngrams_mod, rowvar=False)

# plt.imshow(corrs)
# plt.yticks(np.arange(len(rows)), rows.expl.values)
# plt.xticks(np.arange(len(rows)), rows.expl.values, rotation=90)



df = pd.DataFrame(corrs)
val = rows.expl + " (" + rows.module_num.astype(str) + ")"
df.columns = val
df.index = val
sns.heatmap(df, annot=True)
# sns.clustermap(df, cmap='RdBu_r', center=0, figsize=(20, 20))

plt.show()

In [ ]:

pairs = [
    # related (>0.4)
    (212, 171), # time, measurements

    # medium (~0.2)
    (339, 337), # physical injury or trauma, moments
    (426, 442), # laughter, birthdays

    # unrelated (< -0.3)
    (122, 299), # locations, communication
    (398, 79), # emotional expression, food preparation
    
]

In [ ]:
examples_one = rows.iloc[:2]
expls_one = examples_one['expl'].values
# examples_list_one = examples_one['top_ngrams_module_correct'].values
examples_list_one = examples_one['top_ngrams_module_correct'].apply(lambda x: x[:3]).values

examples_two = rows.iloc[2:4]
expls_two = examples_two['expl'].values
# examples_list_two = examples_two['top_ngrams_module_correct'].values
examples_list_two = examples_two['top_ngrams_module_correct'].apply(lambda x: x[:3]).values

In [ ]:
examples_list_one

In [ ]:
version = 'v0'
prompts = sasc.generate_helper.get_prompts_interaction(
    expls_one,
    expls_two,
    examples_list_one,
    examples_list_two,
    version=version,
)

PV = notebook_helper.get_prompt_templates(version)

In [ ]:
paragraphs = imodelsx.sasc.llm.get_paragraphs(
    prompts,
    checkpoint="gpt-4-0314",
    prefix_first=PV["prefix_first"],
    prefix_next=PV["prefix_next"],
    cache_dir="/home/chansingh/cache/llm_stories",
)

In [ ]:
len(paragraphs)

In [ ]:
for para in paragraphs:
    pprint(para)
    print()